In [180]:
import json
#If pandas give you an error make sure you have installed the package using 'pip install pandas' in the command line
import pandas as pd
import urllib.request

from datetime import datetime



In [233]:
#Replace with your access token, remember the regular token only last for 30 minutes
#You can get your token here: https://developers.facebook.com/tools/explorer/

my_token = 'EAACEdEose0cBAC4aHXbnmbnmJHGjnkjhEwYEmQx20Iqyl10aZCgFQ8sZC28eta1OFCBosZBhuhglYnBrmTetroXJqPKtzOZBBTnIZBz2u51pPkXay0qi4SZAN8yUZD'
date_pat = '%Y-%m-%d'

#Convert date to EPOCH format
'''
    Replace date in format yyyy-mm-dd
'''
since_date = datetime.strptime('2017-01-01',date_pat).strftime('%s') 
until_date = datetime.strptime('2017-03-01',date_pat).strftime('%s')

#Fanpage ID, replace it with id of competetitor you wish to track
fanpage = '349930856069'


In [234]:
#Setting Parameters
data = urllib.parse.urlencode(
    {
        'access_token': my_token,
        'fields' : 'created_time,comments.limit(0).summary(true),likes.limit(0).summary(true),shares',
        'since': since_date,
        'until': until_date,
        'limit': 100
    }
)
print(data)
fields = 'comments.limit(0).summary(true),likes.limit(0).summary(true),shares'

access_token=EAACEdEose0cBAC4aHXAMZByOVl8IOqIbUEKiaEwYEmQx20Iqyl10aZCgFQ8sZC28eta1Op6bMsPvhglNLTNE6lzZAk2p2RHQdDFCBosZBhuhglYnBrmTetroXJqPKtzOZBBTrQYE5h9oLBna6JeoTduy6rEOoeWNdIBS8WN3dnIZBz2u51pPkXay0qi4SZAN8yUZD&until=1488344400&since=1483246800&limit=5&fields=created_time%2Ccomments.limit%280%29.summary%28true%29%2Clikes.limit%280%29.summary%28true%29%2Cshares


In [235]:
#Getting data from Facebook
res = urllib.request.urlopen("https://graph.facebook.com/v2.8/%s/posts?%s&" % (fanpage,data)).read()
res_dict = json.loads(res.decode("ascii"))

In [236]:
posts_array = res_dict['data']

#itereting over paging
while 'paging' in res_dict:
    res = urllib.request.urlopen(res_dict['paging']['next']).read()
    res_dict = json.loads(res.decode("ascii"))
    posts_array += res_dict['data']

In [237]:
#Cleaning data likes, comments and shares
for post in posts_array:
    post['comments_count'] = post['comments']['summary']['total_count']
    post['likes_count'] = post['likes']['summary']['total_count']
    if 'shares' in post:
        post['shares_count'] = post['shares']['count']
    post.pop('likes', None)
    post.pop('comments', None)
    post.pop('shares', None)

In [238]:
#FillNA with 0s
posts_df = pd.DataFrame(posts_array)
posts_df.shares_count.fillna(value=0, inplace=True)

In [239]:
#posts_df['created_time'] = posts_df.created_time.apply(lambda date:datetime.strptime(date,"%Y-%m-%dT%H:%M:%S+%f"))

#Get Month and Year from created_time
posts_df['year'] = pd.DatetimeIndex(posts_df.created_time).year
posts_df['month'] = pd.DatetimeIndex(posts_df.created_time).month

posts_df['activity'] = posts_df['likes_count'] + posts_df['comments_count'] + posts_df['shares_count']

In [240]:
posts_df

,comments_count,created_time,id,likes_count,shares_count,year,month,activity
0,11,2017-02-20T18:00:02+0000,349930856069_10154885534431070,33,0.0,2017,2,44.0
1,37,2017-02-20T14:16:09+0000,349930856069_10154890292066070,126,4.0,2017,2,167.0
2,28,2017-02-17T18:45:00+0000,349930856069_10154882081306070,431,7.0,2017,2,466.0
3,39,2017-02-16T18:49:01+0000,349930856069_10154878176551070,132,2.0,2017,2,173.0
4,7,2017-02-14T18:00:01+0000,349930856069_10154870638371070,63,1.0,2017,2,71.0
5,15,2017-02-13T21:00:00+0000,349930856069_10154868403726070,70,8.0,2017,2,93.0
6,158,2017-02-13T18:07:10+0000,349930856069_10154868125511070,299,16.0,2017,2,473.0
7,15,2017-02-10T18:02:57+0000,349930856069_10154860318501070,121,1.0,2017,2,137.0
8,32,2017-02-09T19:00:38+0000,349930856069_10154857736356070,98,2.0,2017,2,132.0
9,20,2017-02-08T14:30:32+0000,349930856069_10154854224376070,81,3.0,2017,2,104.0


In [252]:
#Summarize by month
summary_df = posts_df.groupby('month').sum()[['likes_count','comments_count','shares_count','activity']]
summary_df['total_posts'] = posts_df.groupby('month').count()['id']

#Transpose Dataframe
print("Activity for %s fanpage" % (fanpage,))
summary_df.T

Activity for 349930856069 fanpage


month,1,2
likes_count,2970.0,2050.0
comments_count,768.0,502.0
shares_count,83.0,56.0
activity,3821.0,2608.0
total_posts,24.0,15.0
